In [1]:
# import numpy as np
# import matplotlib.pyplot as plt
import torch
import torch.nn.functional as F
import torch.nn as nn
from torchtext.legacy import data, datasets

In [12]:
BATCH_SIZE = 100
LR = 0.001
EPOCHS = 5
USE_CUDA = torch.cuda.is_available()

In [13]:
DEVICE = torch.device("cuda" if USE_CUDA else "cpu")
TEXT = data.Field(sequential=True, batch_first=True, lower=True)
LABEL = data.Field(sequential=False, batch_first=True)
trainset, testset = datasets.IMDB.splits(TEXT, LABEL)

In [14]:
# 문장을 단어로 쪼개서 train dataset 안에서 몇 개가 있는지 
TEXT.build_vocab(trainset, min_freq=5)  # 최소 5번 이상 나온 단어만 넣어라
LABEL.build_vocab(trainset)

In [15]:
trainset, valset = trainset.split(split_ratio=0.8)
train_iter, val_iter, test_iter = data.BucketIterator.splits((trainset, valset, testset), batch_size=BATCH_SIZE, shuffle=True, repeat=True)

In [16]:
vocab_size = len(TEXT.vocab)
n_classes = 2
print(f"[TRAIN]:{len(trainset)} \t [VALID]:{len(valset)} \t [TEST]:{len(testset)} \t [VOCAB]:{vocab_size} \t [CLASSES]:{n_classes}")

[TRAIN]:20000 	 [VALID]:5000 	 [TEST]:25000 	 [VOCAB]:46159 	 [CLASSES]:2


In [24]:
class BasicRNN(nn.Module):
    def __init__(self, n_layers, hidden_dim, n_vocab, embed_dim, n_classes, dropout_p=0.2):
        super(BasicRNN, self).__init__()
        print("Building RNN")
        self.n_layers = n_layers
        
        # 보통 n_vocab 단어수가 엄청 많아서 one-hot-encoding 불가능
        # n_vocab x embed_dim (한 단어를 몇 차원으로 표현할 것인지) 사이즈의 matrix 를 만들어서 사용해보자
        self.embed = nn.Embedding(n_vocab, embed_dim)

        self.hidden_dim = hidden_dim
        self.dropout = nn.Dropout(dropout_p)
        self.rnn = nn.RNN(embed_dim, self.hidden_dim, num_layers=self.n_layers, batch_first=True)
        self.out = nn.Linear(self.hidden_dim, n_classes)
    
    def forward(self, x):
        x = self.embed(x)
        h_0 = self._init_state(batch_size=x.size(0))
        x, _ = self.rnn(x, h_0)
        h_t = x[:, -1, :]
        self.dropout(h_t)
        logit = torch.sigmoid(self.out(h_t))
        return logit

    def _init_state(self, batch_size=1):
        weight = next(self.parameters()).data
        return weight.new(self.n_layers, batch_size, self.hidden_dim).zero_()


In [18]:
def train(model, optimizer, train_iter):
    model.train()
    for b, batch in enumerate(train_iter):
        x, y = batch.text.to(DEVICE), batch.label.to(DEVICE)
        y.data.sub_(1)
        optimizer.zero_grad()

        logit = model(x)
        loss = F.cross_entropy(logit, y)
        loss.backward()
        optimizer.step()

        if b%50 == 0:
            print(f"Train Epoch: {e} [{b*len(x)}/{len(train_iter.dataset)} ({100*b/len(train_iter):.0f}%)]\t Loss:{loss.item():.6f}")

In [19]:
def evaluate(model, val_iter):
    model.eval()
    corrects, total_loss = 0, 0

    for batch in val_iter:
        x, y = batch.text.to(DEVICE), batch.label.to(DEVICE)
        y.data.sub_(1)
        logit = model(x)
        loss = F.cross_entropy(logit, y, reduction="sum")
        total_loss += loss.item()
        corrects += (logit.max(1)[1].view(y.size()).data == y.data).sum()

    size = len(val_iter.dataset)
    avg_loss = total_loss / size
    avg_accuracy = 100.0 * corrects / size
    return avg_loss, avg_accuracy

In [26]:
model = BasicRNN(n_layers=1, hidden_dim=256, n_vocab=vocab_size, embed_dim=128, n_classes=n_classes, dropout_p=0.5).to(DEVICE)
optimizer = torch.optim.Adam(model.parameters(), lr=LR)
for e in range(1, EPOCHS + 1):
    train(model, optimizer, train_iter)
    val_loss, val_accuracy = evaluate(model, val_iter)
    print(f"[EPOCH: {e}], Validation Loss: {val_loss:5.2f} | Validation accuracy : {val_accuracy:5.2f}")

Building RNN
Train Epoch: 1 [0/20000 (0%)]	 Loss:0.689944


KeyboardInterrupt: 

In [ ]:
test_loss, test_acc = evaluate(model, test_iter)
print(f"Test Loss: {test_loss:5.2} | Test Accuracy: {test_acc:5.2f}")